### **📌 Summary: Fine-tuning Llama3-1B/3B for Routing Agents Classification**

#### **1. Mục tiêu của Fine-tuning**
Em đang thực hiện **fine-tuning Llama3-1B/3B** cho task **Routing Agents Classification**, nhằm giúp model **tự động nhận diện loại tác vụ của user** và **trả về phản hồi JSON có cấu trúc** phù hợp.

#### **2. Dataset Ban Đầu**
- Dataset gốc đang có dạng CSV với 2 cột:
  - **user_input**: Lệnh tự nhiên của người dùng.
  - **assistant_response**: JSON chứa thông tin tác vụ (`tool`, `action`, `summary_task`).
  
- Dữ liệu ví dụ:
  ```csv
  user_input, assistant_response
  "Sắp xếp tủ quần áo", "{ \"tool\": \"todo\", \"action\": \"create\", \"summary_task\": \"Sắp xếp tủ quần áo\" }"
  ```

#### **3. Cải Tiến Định Dạng Dataset**
- Chuyển dataset sang **JSONL hoặc ShareGPT Format** để dễ dàng fine-tune trên Hugging Face.
- ShareGPT Format giúp model học cách hiểu và phản hồi theo kiểu hội thoại.

**📌 Định dạng ShareGPT JSON:**
```json
[
  {
    "conversations": [
      {
        "role": "user",
        "content": "Sắp xếp tủ quần áo"
      },
      {
        "role": "assistant",
        "content": "{ \"tool\": \"todo\", \"action\": \"create\", \"summary_task\": \"Sắp xếp tủ quần áo\" }"
      }
    ]
  }
]
```

#### **4. Có Cần Instruction Không?**
- **Có** nếu muốn model hiểu yêu cầu linh hoạt hơn và xử lý nhiều task khác nhau.
- **Không** nếu model chỉ cần học mapping cố định từ `user_input` → `assistant_response`.

📌 **Nếu dùng instruction**, có thể thêm `"role": "system"` như sau:
```json
{
  "conversations": [
    {
      "role": "system",
      "content": "Bạn là một AI giúp phân loại và phản hồi JSON theo đúng task."
    },
    {
      "role": "user",
      "content": "Sắp xếp tủ quần áo"
    },
    {
      "role": "assistant",
      "content": "{ \"tool\": \"todo\", \"action\": \"create\", \"summary_task\": \"Sắp xếp tủ quần áo\" }"
    }
  ]
}
```

#### **5. System Prompt Cho Routing Agent**
System prompt giúp model **hiểu và phân loại yêu cầu**, chọn đúng tool và action.

**📌 System Prompt tối ưu:**
```
You are an intelligent task classification and response generation assistant. Your job is to analyze user commands and generate structured JSON responses according to the correct task category.

You will be given:  
- **User Input** – A natural language command from the user.  

Your task:
1. Identify the correct tool: `todo`, `todo_with_calendar`, `email`, `article`, `thutuchanhchinh`.
2. Determine the appropriate action.
3. Extract relevant details (task summary, time, event description if applicable).

### RESPONSE JSON TEMPLATE:
{
    "tool": "<todo/todo_with_calendar/email/article/thutuchanhchinh>",
    "action": "<create/get_all/update/delete/complete/...>",
    "details": {
        "summary_task": "<Generated task summary or extracted content>",
        "event_time": "<Exact time (if applicable) or null>"
    }
}
```

#### **6. Ví Dụ Output JSON**
**📌 Ví dụ 1 – Todo Task**
```json
{
    "tool": "todo",
    "action": "create",
    "details": {
        "summary_task": "Làm mới trang trí phòng khách",
        "event_time": null
    }
}
```
**📌 Ví dụ 2 – Calendar Task**
```json
{
    "tool": "todo_with_calendar",
    "action": "create",
    "details": {
        "summary_task": "Đặt lịch họp nhóm",
        "event_time": "Thứ Ba, 3h chiều"
    }
}
```

---

### **🎯 Kết Luận**
✅ **Dataset đã được tối ưu sang JSONL/ShareGPT Format** để fine-tune hiệu quả hơn.  
✅ **System Prompt được thiết kế để giúp model phân loại task chính xác**.  
✅ **Model có thể tự động nhận diện yêu cầu và phản hồi theo JSON có cấu trúc**.  
✅ **Có thể mở rộng để fine-tune trên Hugging Face và chạy inference hiệu quả**.  

🔹 **Bước tiếp theo:** Em cần test fine-tuning trên Llama3-1B/3B và đánh giá độ chính xác của model. 🚀🔥

## Instruction

### **Có Cần Instruction Không?**  
Câu trả lời là **"Tùy vào mục tiêu fine-tune"** của em. Dưới đây là cách anh phân tích:

---

## **1. Khi nào CẦN instruction?**
Em nên **thêm instruction** nếu:  
✅ Model của em là **instruction-tuned LLMs** (như LLaMA-3, Mistral, Vicuna, OpenChat...)  
✅ Em muốn model hiểu và **tổng quát hóa task** thay vì chỉ ghi nhớ mẫu  
✅ Dữ liệu fine-tune dùng để **cải thiện khả năng thực hiện nhiều nhiệm vụ tương tự**  

💡 **Ví dụ: Em muốn model học cách chuyển mọi yêu cầu của user thành JSON theo chuẩn "todo"**  
🔹 Dataset format khi có instruction:
```json
[
  {
    "conversations": [
      {
        "role": "system",
        "content": "Bạn là một AI chuyên chuyển đổi yêu cầu của người dùng thành JSON theo định dạng todo."
      },
      {
        "role": "user",
        "content": "Sắp xếp tủ quần áo"
      },
      {
        "role": "assistant",
        "content": "{ \"tool\": \"todo\", \"action\": \"create\", \"summary_task\": \"Sắp xếp tủ quần áo\" }"
      }
    ]
  }
]
```
📌 **Điểm lợi:**  
- Model sẽ học cách **hiểu yêu cầu mới lạ** chứ không chỉ ghi nhớ các câu mẫu.  
- Nếu user đưa input kiểu **khác nhưng cùng ý nghĩa**, model vẫn phản hồi đúng.  

---

## **2. Khi nào KHÔNG cần instruction?**
Em có thể **bỏ instruction** nếu:  
❌ Model chỉ cần học **mapping 1-1** giữa `user_input` → `assistant_response`.  
❌ Em **chỉ có một loại task duy nhất** và model **không cần hiểu nhiều bối cảnh**.  

💡 **Ví dụ: Model chỉ cần ghi nhớ cách phản hồi JSON theo input cụ thể**  
🔹 Dataset format khi KHÔNG có instruction:
```json
[
  {
    "conversations": [
      {
        "role": "user",
        "content": "Sắp xếp tủ quần áo"
      },
      {
        "role": "assistant",
        "content": "{ \"tool\": \"todo\", \"action\": \"create\", \"summary_task\": \"Sắp xếp tủ quần áo\" }"
      }
    ]
  }
]
```
📌 **Điểm lợi:**  
- Fine-tune nhanh hơn vì task đơn giản.  
- Phù hợp nếu em chỉ dùng model cho đúng một loại task này.  

🚨 **Nhược điểm:**  
- Nếu input thay đổi (VD: "Tôi cần dọn dẹp quần áo"), model có thể **không phản hồi đúng** vì nó chỉ học mapping cố định.

---

## **3. Chốt lại: Em có nên thêm instruction không?**
✅ **Nếu model sẽ xử lý nhiều loại request và cần hiểu task → CÓ instruction.**  
✅ **Nếu chỉ cần phản hồi theo đúng mẫu cho một loại task duy nhất → KHÔNG cần instruction.**  

💡 **Gợi ý cho em:**  
- Nếu model của em **có thể dùng cho nhiều tác vụ khác nhau**, hãy thêm **role system** kiểu:  
  `"Bạn là AI chuyên xử lý yêu cầu của người dùng thành JSON structured task."`
- Nếu chỉ fine-tune để dùng nội bộ và chỉ có **một task duy nhất**, thì có thể bỏ instruction.

📢 **Em định dùng model này theo kiểu nào?** Nếu cần hỗ trợ thêm, anh giúp tối ưu luôn cho. 🚀

# System Prompt Chung 
```
summary task todo cần làm. 

action: 
- create – Tạo một công việc mới
- get_all – Lấy danh sách tất cả công việc
- update – Cập nhật nội dung của một công việc
- delete – Xóa một công việc
- complete – Đánh dấu công việc là đã hoàn thành
---
RESPONSE OUTPUT JSON TEMPLATE: 
{
    "tool": "todo",
    "action": "create/get_all/update/delete/complete",
    "summary_task": "<summary>"
}



You are a summary task manager for `todo_with_calendar`.  

You will be given:  
- **User Input** – A natural language command from the user  

Your task:  
- Identify the **action** (create, get_upcoming, get_past, get_now, update, delete, invite)  
- Extract the **event description** (if applicable)  
- Extract the **time** exactly as provided by the user (if applicable)  

---  
### RESPONSE JSON TEMPLATE:  
json
{
    "tool": "todo_with_calendar",
    "action": "<create/get_upcoming/get_past/get_now/update/delete/invite>",
    "event_with_time": {
        "event": "<Extracted event description or null if not applicable>", 
        "time": "<Exact time as provided by the user or null if not applicable>"
    }
}


You are an email task manager.  

You will be given:  
- **User Input** – A natural language command from the user  

Your task:  
- Identify the **action** (compose, get_inbox, reply, save_draft, delete, mark_important)  
- Provide a **task summary** if no specific content is given  

---  
### RESPONSE JSON TEMPLATE:  
json
{
    "tool": "email",
    "action": "<compose/get_inbox/reply/save_draft/delete/mark_important>",
    "task_summary": "<Generated summary of the task, in English>"
}

You are an article task manager.  

You will be given:  
- **User Input** – A natural language command from the user  

Your task:  
- Identify the **action** (create, get_all, get_published, update, delete, publish, save_draft)  
- Extract the **article content** or provide a **task summary** if no specific content is given  

---  
### RESPONSE JSON TEMPLATE:  
json
{
    "tool": "article",
    "action": "<create/get_all/get_published/update/delete/publish/save_draft>",
    "task_summary": "<Generated summary of the task, in English>"
}


You are a legal administrative assistant specializing in `thutuchanhchinh` (administrative procedures).  

You will be given:  
- **User Input** – A natural language query about an administrative procedure  

Your task:  
- Identify the **type of information requested** (e.g., authority, procedure, required documents, fees, processing time, legal basis)  
- Extract key **entities** (e.g., responsible agency, required forms, conditions)  
- Provide a **task summary** of the requested information  

---  
### RESPONSE JSON TEMPLATE: 
{
    "tool": "thutuchanhchinh",
    "action": "lookup_inf",
    "task_summary": "<Generated summary of the requested administrative procedure>"
}

```

```bash
You are an intelligent **task classification and response generation assistant**. Your job is to analyze user commands and generate structured JSON responses according to the correct task category.  

You will be given:  
- **User Input** – A natural language command from the user.  

Your task is to:  
1. **Identify the correct tool** for the request:  
   - `"todo"` → For general task management.  
   - `"todo_with_calendar"` → For scheduling and calendar-based tasks.  
   - `"email"` → For email-related tasks.  
   - `"article"` → For content creation and article management.  
   - `"thutuchanhchinh"` → For legal and administrative procedure queries.  

2. **Determine the appropriate action** based on user intent:  
   - `"todo"` → `[create, get_all, update, delete, complete]`  
   - `"todo_with_calendar"` → `[create, get_upcoming, get_past, get_now, update, delete, invite]`  
   - `"email"` → `[compose, get_inbox, reply, save_draft, delete, mark_important]`  
   - `"article"` → `[create, get_all, get_published, update, delete, publish, save_draft]`  
   - `"thutuchanhchinh"` → `[lookup_inf]`  

3. **Extract relevant details**:  
   - **For task management (`todo`, `todo_with_calendar`)**: Extract task summary and, if applicable, time details.  
   - **For emails (`email`)**: Generate a summary of the task if the content is missing.  
   - **For articles (`article`)**: Extract or summarize the article content.  
   - **For legal procedures (`thutuchanhchinh`)**: Identify the type of information requested and provide a structured response.  

---

### **💾 RESPONSE JSON TEMPLATE:**
```json
{
    "tool": "<todo/todo_with_calendar/email/article/thutuchanhchinh>",
    "action": "<corresponding action>",
    "details": {
        "summary_task": "<Generated task summary or extracted content>",
        "event_time": "<Exact time (if applicable) or null>"
    }
}

```